# Aplicaciones de Ciencias de la Computación (INF265)
## Laboratorio 1: Agente aspirador de 6 posiciones

Al finalizar el presente laboratorio se debe tener implementado el entorno de trabajo del agente aspirador de 6 posiciones y un programa reflexivo simple para dicho agente. Las posiciones del entorno son denotadas como loc_A, loc_B, loc_C, loc_D, loc_E y loc_F. Cada una de estas posiciones puede tener el estado 'Dirty' o 'Clean'.  
Al final del notebook deberas responder a las preguntas planteadas. 

# Clase <b>Thing</b>

Esta clase generica representa cualquier objeto fisico que puede aparecer en un <b>Ambiente</b>. (No editar)  

In [1]:
class Thing(object):

    def is_alive(self):
        """Cosas 'vivas'deben retornar true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Muestra el estado interno del agente. Subclases deben sobreescribir esto."""
        print("I don't know how to show_state.")

# Clase <b>Agent</b>

Un agente es una subclase de Thing con un slot obligatorio: <b>.program</b>, el cual almacena la funcion que implementa el <b>programa del agente</b>. Esta funcion debe tomar como argumento la <b>percepcion</b> del agente y debe retornar una <b>accion</b>. La definicion de Percepcion y Accion depende del ambiente de trabajo (environment) donde el agente existe. El agente tambien puede tener el slot <b>.performance</b>, que mide el desempeño del agente en su ambiente.

In [2]:
import collections
import random

class Agent(Thing):

    def __init__(self, program=None):
        self.alive = True
        self.performance = 0
        assert isinstance(program, collections.Callable)
        self.program = program

# Clase <b>Environment</b>

Esta clase abstracta representa un entorno de tareas. Clases de entornos reales heredan de esta. En un entorno tipicamente se necesitará implementar 2 cosas:
<b>percept</b>, que define la percepción que el agente ve; y 
<b>execute_action</b>, que define los efectos de ejecutar una acción. 
El entorno mantiene una lista de .things y .agents (el cual es un subconjunto de .things). Cada elemento de .things tiene un slot .location. (No editar)

In [3]:
class Environment(object):

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Retorna la percepción que el agente 'agent' ve en este punto."""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """El agente 'agent' ejecuta una acción 'action' en el entorno."""
        raise NotImplementedError

    def default_location(self, thing):
        """Localización por defecto para colocar una nueva cosa sin localización especificada."""
        return None

    def is_done(self):
        """Retorna True si no hay ningón agente vivo"""
        return not any(agent.is_alive() for agent in self.agents)

    def add_thing(self, thing, location=None):
        """Añade una cosa thing al entorno en la localización location. 
           Si thing es un programa de agente, crea un nuevo agente con ese programa."""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        assert thing not in self.things, "No añade la misma cosa dos veces"
        thing.location = location if location is not None else self.default_location(thing)
        self.things.append(thing)
        if isinstance(thing, Agent):
            thing.performance = 0
            self.agents.append(thing)

    def step(self):
        """Ejecuta un paso del entorno (llama a los programas de los agentes, obtiene sus acciones y las ejecuta). """
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)

    def run(self, steps=1000):
        """Ejecuta steps pasos en el entorno."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()

#### Clase <b>VacuumEnvironment</b>

Esta clase implementa el entorno del aspirador de 6 posiciones: loc_A, loc_B, loc_C, loc_D, loc_E y loc_F. Cada una de estas posiciones puede tener el estado 'Dirty' o 'Clean'. Un agente en este entorno percibe su localizacion y el estado de la misma

In [44]:

# Considere el siguiente ambiente:

#############
# A # B # C #
#############
# D # E # F #
#############
# Complete las posiciones reemplazando los guiones bajos por números
loc_A, loc_B, loc_C, loc_D, loc_E, loc_F = (0,0), (0,1), (0, 2), (1, 0), (1, 1), (1, 2)

class VacuumEnvironment(Environment):

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty']),
                       loc_C: random.choice(['Clean', 'Dirty']),
                       loc_D: random.choice(['Clean', 'Dirty']),
                       loc_E: random.choice(['Clean', 'Dirty']),
                       loc_F: random.choice(['Clean', 'Dirty']),}

    def thing_classes(self):
        return [ReflexVacuumAgent]

    def percept(self, agent):
        """Retorna la posición del agente y el estado de la posición (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Implementa el MAPA DE TRANSICIÓN: Cambia la posición del agente y/o el estado de la posición; 
        Cada aspiración (acción 'suck') en una localización Dirty provoca un aumento de desempeño en 10 unidades;
        Cada movida efectiva Right, Left, Up o Down provoca una disminución de desempeño en 1 unidad """
        
        if action == 'Suck':
            self.status[agent.location] = 'Clean'
            agent.performance += 10
        else:
            # Completar los movimientos para el agente
            # Debes identificar la direccion de la accion y modificar acordemente el x o y
            # Pista: Ten cuidado con los limites del ambiente!
            actual_x, actual_y = agent.location

            # Complete su codigo aqui
            # Solución a la pregunta 1. El else último que añadí se puede quitar y solo penalizar cuando se mueve.
            # Sin embargo, lo dejé de ahí asumiendo que los evaluadores no quieren que se modifique el código.
            # Igual cumple con su cometido. La solución a la pregunta 3 con el código modificado la he colocado
            # como un bloque separado junto con las preguntas de la sección inferior.
            delta_x, delta_y = 0, 0
            if action == 'Down':
                delta_x = 1
            elif action == 'Left':
                delta_y = -1
            elif action == 'Right':
                delta_y = 1
            elif action == 'Up':
                delta_x = -1
            next_x, next_y = actual_x + delta_x, actual_y + delta_y
            # Si no nos hemos salido del tablero, realizamos un movimiento efectivo
            if 0 <= next_x and next_x <= 1 and 0 <= next_y and next_y <= 2:
                actual_x += delta_x
                actual_y += delta_y
            else:
                # Si nos salimos del tablero, como no realizaremos un movimiento efectivo, le sumo uno para
                # neutralizar la penalización.
                agent.performance += 1
            agent.location = actual_x, actual_y
            agent.performance -= 1

    def default_location(self, thing):
        """Devuelve una posicion aleatoria."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

# Agente Aspirador de 6 posiciones con Programa Reactivo Simple

Este agente es el agente aspirador de cuatro posiciones que usa un programa reactivo simple: realiza una accion basado en la percepción (posicion, estado) actual

In [18]:
def ReflexVacuumAgent():
    
    def program(percept):
        location, status = percept
        if status == 'Dirty':
            return 'Suck'
        elif location == loc_A:
            return 'Right'
        elif location == loc_B:
            return 'Right'
        elif location == loc_C:
            return 'Down'
        elif location == loc_D:
            return 'Up'
        elif location == loc_E:
            return 'Left'
        elif location == loc_F:
            return 'Left'
    return Agent(program)

# Probando el agente reflexivo simple en su entorno

In [50]:
"""Crea el entorno del aspirador de 6 posiciones con 2 posiciones en estado 'Dirty'"""
e = VacuumEnvironment()
e.status = {loc_A: 'Clean',  loc_B: 'Dirty', loc_C: 'Clean', loc_D: 'Dirty', loc_E: 'Clean', loc_F: 'Clean'}

"""Crea un agente reflexivo simple"""
a = ReflexVacuumAgent()

"""Añade el agente creado al entorno en posicion loc_A"""
e.add_thing(a, location=loc_A) 

# Imprime el estado inicial del ambiente y localizacion del agente
print("Estado Inicial del Ambiente: {}".format(e.status))
print("ReflexVacuumAgent esta localizado en {} con desempeño = {}".format(a.location, a.performance))

"""Ejecuta el entorno 6 pasos y obtiene el desempeño del agente"""
e.run(6)

# Imprime el estado actual del ambiente, localizacion del agente y su desempeño
print("Estado del Ambiente despues de 6 pasos: {}".format(e.status))
print("ReflexVacuumAgent esta localizado en {} con desempeño = {}".format(a.location, a.performance))


Estado Inicial del Ambiente: {(0, 0): 'Clean', (0, 1): 'Dirty', (0, 2): 'Clean', (1, 0): 'Dirty', (1, 1): 'Clean', (1, 2): 'Clean'}
ReflexVacuumAgent esta localizado en (0, 0) con desempeño = 0
Estado del Ambiente despues de 6 pasos: {(0, 0): 'Clean', (0, 1): 'Clean', (0, 2): 'Clean', (1, 0): 'Dirty', (1, 1): 'Clean', (1, 2): 'Clean'}
ReflexVacuumAgent esta localizado en (1, 0) con desempeño = 7


# Preguntas:
1. Completar el codigo en la clase Vacuum Enviroment. No cambiar la medida de desempeño que tiene el ambiente, sólo completar la lógica de movimientos. (**4 pts**)

Completado. Se obtuvo una medida de desempeño de 5.

2. Después de haber completado el código, ¿puede plantear un programa de Agente que maximice de mejor manera  la medida de desempeño implementada en el entorno?  No es necesario modificar el código, sino que puede explicarlo en sus propias palabras. (4 pts) (**4 pts**)

Primero, en vez de penalizar el movimiento deberíamos penalizar la no limpieza al llegar a una casilla puesto que nuestra función quiere medir el éxito de limpiar. Luego, En ReflexVacuumAgent tenemos que se mueve en sentido horario, lo cual es una manera determinísitca de hacerlo. Realizar el movimiento de manera aleatoria sería mucho mejor ya que realmente la distribución de la suciedad en nuestro ambiente también estará distribuido de manera aleatoria. Para esto, en vez de manualmente con if's decidir el movimiento, podríamos realizar un random.choice. Se necesitan más matemáticas para probar que el valor esperado del número de movimientos cuando nos dirigimos en una dirección de manera aleatoria cuando el mapa está Sucio / Limpio inicialmente de manera aleatoria, debería ser menor que el valor esperado cuando nuestro movimiento está perfectamente determinado. Asimismo, en el entorno sería mejor poder modificar la percepción ya que, en un caso hipotético en el cual solo tengamos una suciedad en loc_D, en vez de dar toda la vuelta, si tan solo nos dirigiésemos hacia él y lo limpiamos, todo estaría acabado antes con un mejor desempéño en vez de tener que recorrer todo, en este tipo de casos, realizar una acción greedy nos ayudaría.

3. Basado en el programa de agente original, ¿que debería cambiar en el entorno para que el programa de agente maximize la medida de desempeño? Modifique el código. (**4 pts**)

Para maximizar la medida de desempeño, en vez de penalizar por el movimiento, deberíamos penalizar si es que la aspiradora limpió o no limpió en el lugar al cual ha llegado. Aquí está el código modificado, el cual corriendo el bloque de prueba obtuvo un desempeño de 7, superior al desempeño 5 de la pregunta 1.

In [46]:

# Considere el siguiente ambiente:

#############
# A # B # C #
#############
# D # E # F #
#############
# Complete las posiciones reemplazando los guiones bajos por números
loc_A, loc_B, loc_C, loc_D, loc_E, loc_F = (0,0), (0,1), (0, 2), (1, 0), (1, 1), (1, 2)

class VacuumEnvironment(Environment):

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty']),
                       loc_C: random.choice(['Clean', 'Dirty']),
                       loc_D: random.choice(['Clean', 'Dirty']),
                       loc_E: random.choice(['Clean', 'Dirty']),
                       loc_F: random.choice(['Clean', 'Dirty']),}

    def thing_classes(self):
        return [ReflexVacuumAgent]

    def percept(self, agent):
        """Retorna la posición del agente y el estado de la posición (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Implementa el MAPA DE TRANSICIÓN: Cambia la posición del agente y/o el estado de la posición; 
        Cada aspiración (acción 'suck') en una localización Dirty provoca un aumento de desempeño en 10 unidades;
        Cada movida efectiva Right, Left, Up o Down provoca una disminución de desempeño en 1 unidad """
        
        if action == 'Suck':
            self.status[agent.location] = 'Clean'
            agent.performance += 10
        else:
            # Completar los movimientos para el agente
            # Debes identificar la direccion de la accion y modificar acordemente el x o y
            # Pista: Ten cuidado con los limites del ambiente!
            actual_x, actual_y = agent.location

            # Complete su codigo aqui
            # Solución a la pregunta 3. Como queremos medir el desempeño de limpieza, lo que penalizaré será el hecho
            # de que nuestro aspirador limpie o no el suelo.
            delta_x, delta_y = 0, 0
            if action == 'Down':
                delta_x = 1
            elif action == 'Left':
                delta_y = -1
            elif action == 'Right':
                delta_y = 1
            elif action == 'Up':
                delta_x = -1
            next_x, next_y = actual_x + delta_x, actual_y + delta_y
            # Si no nos hemos salido del tablero, entonces el movimiento es válido
            if 0 <= next_x and next_x <= 1 and 0 <= next_y and next_y <= 2:
                actual_x += delta_x
                actual_y += delta_y
                agent.location = actual_x, actual_y
                # Como la posicion del tablero es valida, entonces puedo verificar si es que está limpia o sucia
                # Voy a penalizar si llegué a una posición limpia ya que no voy a realizar ninguna limpieza ahí
                # en la siguiente acción.
                if self.status[agent.location] == 'Clean':
                    agent.performance -= 1

    def default_location(self, thing):
        """Devuelve una posicion aleatoria."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

4. Según el entorno, ¿existe la posibilidad de que el agente obtenga un desempeño sea negativo? En caso de que su respuesta sea positiva, explique como sería dicho escenario. (**4 pts**)

Sí, existen muchas maneras de conseguirlo. Una manera sería que no le demos los pasos suficientes, por ejemplo, realizados n pasos, tenemos que nuestra función de desempeño sería f:N -> Z con f(n) = 10 * (# Sucks) - (# Pasos). En la distribución dada en el ejemplo tenemos

Clean Dirty Clean

Dirty Clean Clean

En este caso particular notamos que si hacemos que ejecute el entorno tan solo 1 paso, nuestro desempeño sería -1.
Asimismo, pasados los 23 pasos, nuestra función es negativa y decreciente puesto que todo ya está limpio (con el movimiento horario determinístico que se dio inicialmente).

Finalmente, si el tablero hubiese estado completamente limpio

Clean Clean Clean

Clean Clean Clean

hubiésemos tenido siempre resultados negativos con el programa original de la pregunta 1.

5. ¿Qué característica tendria que tener el entorno para necesitar un agente reactivo basado en modelo? (**4 pts**)

Tendría que ser al menos un entorno parcialmente observable, es decir, depende del estado actual y no de toda la historia. En nuestro caso, la decisión se toma respecto a moverse en el mapa dependiendo de la posición, podríamos hacer que el estado de las casillas adyacentes a nosotros también influyan en nuestra toma de decisión, la cosa es que en nuestra toma de decisión no importó la historia sino como se encontraba el tablero a la hora de tomar una decisión entre limpiar o moverse.